# Критерий остановки

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.graph_objects as go

from collections import namedtuple

np.random.seed(7)

Можно останавливать эксперимент при достижении 95% уверенности, что целевая метрики одной группы лучше других.  
Отметка 95% произвольна. Можно останавливать при 90%. Можно рассмотреть более объективный критерий остановки.

Можно придумать разные способы оценки эффекта при остановке.  
Ожидаемый выигрыш. Или потери в случае неверного решения.  
Как выбрать порог? 
Т.е. проблема та же, что 95% или 90%.  
Более удобный вопрос - есть ли смысл продолжать дальше?

Качественные соображения:  
Каждый день эксперимента - потери за счет попадания в худшую группу.  
Так же потери на поддержку в виде техдолга (сложно оценить).  

Каждый день эксперимента - новые данные и уточнение оценок метрик.  
Ценность данных в начале эксперимента и с течением времени неодинакова.  
Добавление первых 100 точек дает много информации, добавление 100 точек к 10 млн. ничего не меняет в оценке.  

Важны последствия выбора.    
При тестировании лекарств нужна большая уверенность в отсутствии побочек. Высока стоимость ошибки.  
В веб-сервисах выбранных вариант продержится, например, год до повторного или следующего эксперимента.  
Решение об остановке должно учитывать последствия выбора.  

В итоге цена поддержания эксперимента постоянна, ценность новых данных падает.  
Когда ценность новых данных падает ниже стоимости поддержания эксперимента и выгоды/потерь после прекращения,
эксперимент пора останавливать.

Нужно привести эти соображения к одной шкале - денежной.

Нужен пример.  
Пусть есть 2 группы.  
Конвресия 5% и 10%.  
И есть трафик 10000 в день. 
В какой момент останавливать тест?

В одной группе ожидается 1000 сконвертировавшихся в день, в другой 500. 

Лучше так.  
2 группы.  
Насобирали данных.  
Есть оценки конверсий и разности.  
Останавливать или нет?

Нужно посчитать ожидаемый выигрыш, если остановить сейчас.  
И сравнить его с ожидаемым выигрышем, если продолжить.

Как посчитать ожидаемый выигрыш?

Есть распределение разности конверсий.  

При немедленной остановке:  
Посчитать ожидаемое значение.  
Домножить на количество пользователей.  
До след. эксп. ничего меняться не будет - пользователей до след. эксп.  

При продолжении:  
Моделировать сценарии.  
Добавить N точек, посчитать ожидаемую выгоду при остановке. 

Кажется, если перебирать все сценарии, оценка разности не должна меняться.   
Или все-таки будет уточняться?  

Оценка при продолжении.   
Дисперсия апостериорного распределения убывает ~ 1/N.  
Считать по мере набора точек апостериорное будет сужаться к текущему среднему.  

Как тогда строить оценку?
Добавим N точек. Среднее то же, дисперсия меньше.  
Можно обновить ожидаемую выгоду.  
Учесть среди использованных N точек деление между группами.

Пусть есть 2 группы.  
Конверсия одной 10%.  
Другой на 5% больше.  
В каждой пришло 10000 точек.  
Построены апостериорные распределения и разность.  

Если ожидаемое значение не меняется, оценка выгоды после остановки тоже не поменяется.  
Что даст уменьшение дисперсии?

Нужно смотреть куски распределения меньше 0.  
Примерно как потери в случае выбора не той группы.

Пусть $P(p_B > p_A) \equiv P_B$ - вероятность метрика $B$ больше $A$.  
$G$ - дополнительная выгода при выборе $B$ и группа $B$ на самом деле лучше.  
$L$ - потери при выборе $B$ если группа $B$ на самом деле хуже.

$E[U | B]$ - ожидаемая выгода при выборе $B$.

$$
E[U ∣ B]= P_B G - (1-P_B) L 
$$


При выборе $A$ ничего не меняется
$$
E[U | A] = 0
$$

Можно выбирать $B$ если 
$$
P_B G - (1-P_B) L > 0
\\
P_B (G + L) > L
\\
P_B > \frac{L}{G + L}
$$

Как оценить L и G?
Для конверсий в оплату:  
$$
G = E[p_B - p_A | p_B > p_A] * LTV * N  
\\
L = E[-(p_B - p_A) | p_B < p_A] * LTV * N  
$$

$$
P_B > \frac{E[-(p_B - p_A) | p_B < p_A] }{E[p_B - p_A | p_B > p_A] + E[-(p_B - p_A) | p_B < p_A]}
$$

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))


a, b = (0 - approx_diff_dist.mean()) / approx_diff_dist.std(), np.inf
tr_gain = stats.truncnorm(a=a, b=b, loc=approx_diff_dist.mean(), scale=approx_diff_dist.std())

a, b = -np.inf, (0 - approx_diff_dist.mean()) / approx_diff_dist.std()
tr_loss = stats.truncnorm(a=a, b=b, loc=approx_diff_dist.mean(), scale=approx_diff_dist.std())


x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='$\mbox{Аналитическое приближение}$'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.add_trace(go.Scatter(x=x, y=tr_gain.pdf(x), 
                         line_color='black', name='tr-gain'))
fig.add_trace(go.Scatter(x=x, y=tr_loss.pdf(x), 
                         line_color='black', name='tr-loss'))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()


dist_p_b_gt_a = 1 - approx_diff_dist.cdf(0)
print(f'pb > pa: {dist_p_b_gt_a}')

e_gain = tr_gain.mean() * dist_p_b_gt_a
print(f'Egain: {e_gain}')

e_loss = tr_loss.mean() * (1 - dist_p_b_gt_a)
print(f'Eloss: {e_loss}')

c = -tr_loss.mean() / (tr_gain.mean() + -tr_loss.mean())
print(f'Threshold: {c}')

Сравниваются площади.  
Сравниваются средние в кусках распределения.  
Они же связаны. Странно.    

Соотношение
$$
P_B G - (1-P_B) L > 0
$$
ведь то же, что 
$$
E[p] > 0
$$

Поэтому выполнено при любых sigma.  

Нужно учитывать внешние факторы.

Как меняется в зависимости от sigma?

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)


dplt = []
for s in np.arange(0.001, 10, 0.01):
    approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                                  scale=s)
    a, b = (0 - approx_diff_dist.mean()) / approx_diff_dist.std(), np.inf
    tr_gain = stats.truncnorm(a=a, b=b, loc=approx_diff_dist.mean(), scale=approx_diff_dist.std())
    a, b = -np.inf, (0 - approx_diff_dist.mean()) / approx_diff_dist.std()
    tr_loss = stats.truncnorm(a=a, b=b, loc=approx_diff_dist.mean(), scale=approx_diff_dist.std())
    dist_p_b_gt_a = 1 - approx_diff_dist.cdf(0)
    #print(f'pb > pa: {dist_p_b_gt_a}')
    #e_gain = tr_gain.mean() * dist_p_b_gt_a
    #print(f'Egain: {e_gain}')
    #e_loss = tr_loss.mean() * (1 - dist_p_b_gt_a)
    #print(f'Eloss: {e_loss}')
    c = -tr_loss.mean() / (tr_gain.mean() + -tr_loss.mean())
    #print(f'Threshold: {c}')
    dplt.append((s, dist_p_b_gt_a, c))

df = pd.DataFrame(dplt, columns=['s', 'pba', 'c'])
df


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['s'], y=df['pba'], 
                         line_color='blue', name='pba'))
fig.add_trace(go.Scatter(x=df['s'], y=df['c'], 
                         line_color='black', name='c'))
fig.add_trace(go.Scatter(x=df['s'], y=1 - df['c'], 
                         line_color='green', name='1-c'))
# fig.update_layout(title='$p_B - p_A$',
#                   xaxis_title='$x$',
#                   yaxis_title='Плотность вероятности',
#                   xaxis_range=[-0.1, 0.1],
#                   hovermode="x",
#                   height=500)
fig.show()

Для текущих средних и дисперсии можно оценить изменение вероятности.  
Среднее предпологается таким же.  
Дисперсия уменьшается 1/N.  
Можно построить оценку P(pb>pa) от N.  

Можно сравнить со стоимостью данных.  
Для этого нужно считать разницу между шагами.  
И сравнить со стоимостью данных.  

Дисперсия бета-распределения $B(x; \alpha, \beta)$:

$$
Var(x) = \frac{\alpha \beta}{(\alpha + \beta)^2(\alpha + \beta + 1)}
$$

При $N > n_s \gg \alpha_0, \beta_0$:

$$
\alpha \approx n_s, \qquad \beta \approx N - n_s, \qquad n_s \approx p N
\\
Var(x) = \frac{n_s (N - n_s)}{N^3} = \frac{p (1 - p)}{N}
$$

Приближенно можно построить $Var(x) \sim 1/N$.  
Более точно можно промоделировать набор новых точек из правдоподобия и апостериорного.  

Считать время на изменение решения.  
Готовы ли держать еще столько времени?  
Это способ оценить расходы на поддержание эксперимента на глаз.  

Есть $Var(x) \sim 1/N$.  
Есть $P(p_b > p_a)$ в зависимости от N.  
В какой момент останавливаться?  
Нужно сравнить прирост пользы данных с ценой эксперимента.  

https://en.wikipedia.org/wiki/Expected_value_of_sample_information

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))


x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='$\mbox{Аналитическое приближение}$'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()


# pa = sa / na
# pb = sb / nb

# for N in range(0, 10e6, 1000):
#     na_mod = na + N
#     nb_mod = nb + N
#     sa_mod = sa + int(pa * N)
#     sb_mod = sb + int(pb * N)
#     a_mod = stats.beta(a=sa_mod, b=na_mod - sa_mod)
#     b_mod = stats.beta(a=sb_mod, b=nb_mod - sb_mod)
#     diff_mod = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
#                               scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))

res = []
for N in range(na+nb, 100_000, 100):
    s = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)
    p = 1 - stats.norm(loc=approx_diff_dist.mean(), scale=s).cdf(0)
    res.append((N, s, p))
    
#res = [(N, approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)) for N in range(na+nb, 1000000, 100)]

df = pd.DataFrame(res, columns=['N', 's', 'p'])
df


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['s'], 
                         line_color='blue', name='s'))
fig.add_trace(go.Scatter(x=df['N'], y=df['p'], 
                         line_color='black', name='p'))
fig.show()

При одинаковом среднем меньше ожидаемых потерь лучше.  
Останавливать, если:  
Ожидаемые потери на след. шаге - Ожидаемые потери сейчас < Стоимости данных  
Стоимость данных - ожидаемые потери из-за попадания части пользователей в худшую группу.  
Есть еще вероятность, что группа хуже. Но это малый вклад.  

$$
E[L(\delta | N + \Delta N)] - E[L(\delta | N)] < Cost(\Delta N)
\\
E[L(\delta | N)] = LTV * M \int dx \min(0, P_{\delta(N)}(x)) = LTV * M \int_{-\infty}^0 P_{\delta(N)}(x) dx
\approx LTV * M \int_{-\infty}^0 Norm(x; \mu_B(N) - \mu_A(N), \sigma_A^2(N) + \sigma_B^2(N)) dx
\\
Cost(\Delta N) \approx w_A E[\delta] \Delta N / M = w_A (\mu_B - \mu_A) \Delta N / M
$$

$$
E[L(\delta | N + \Delta N)] - E[L(\delta | N)] < (\mu_B(N) - \mu_A(N)) \Delta N / M
\\
(E[L(\delta | N + \Delta N)] - E[L(\delta | N)]) / \Delta N < (\mu_B(N) - \mu_A(N)) / M
\\
 M (E[L(\delta | N)])' < (\mu_B(N) - \mu_A(N))
\\
(E[L(\delta | N)])' < (\mu_B(N) - \mu_A(N)) / M 
$$

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))

a, b = -np.inf, (0 - approx_diff_dist.mean()) / approx_diff_dist.std()
tr_loss = stats.truncnorm(a=a, b=b, loc=approx_diff_dist.mean(), scale=approx_diff_dist.std())

x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='Diff'))
fig.add_trace(go.Scatter(x=x, y=tr_loss.pdf(x), 
                         line_color='red', name='Loss'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()


# pa = sa / na
# pb = sb / nb

# for N in range(0, 10e6, 1000):
#     na_mod = na + N
#     nb_mod = nb + N
#     sa_mod = sa + int(pa * N)
#     sb_mod = sb + int(pb * N)
#     a_mod = stats.beta(a=sa_mod, b=na_mod - sa_mod)
#     b_mod = stats.beta(a=sb_mod, b=nb_mod - sb_mod)
#     diff_mod = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
#                               scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))

dN = 100

res = []
for N in range(na+nb, 100_000, dN):
    mu = approx_diff_dist.mean()
    s = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)
    new_diff_dist = stats.norm(loc=approx_diff_dist.mean(), scale=s)
    p = 1 - new_diff_dist.cdf(0)
    a, b = -np.inf, (0 - new_diff_dist.mean()) / new_diff_dist.std()
    tr_loss = stats.truncnorm(a=a, b=b, loc=new_diff_dist.mean(), scale=new_diff_dist.std())
    mean_loss = tr_loss.mean() * new_diff_dist.cdf(0)
    s2 = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N+dN)
    new_diff_dist2 = stats.norm(loc=approx_diff_dist.mean(), scale=s2)
    a2, b2 = -np.inf, (0 - new_diff_dist2.mean()) / new_diff_dist2.std()
    tr_loss2 = stats.truncnorm(a=a2, b=b2, loc=new_diff_dist2.mean(), scale=new_diff_dist2.std())
    mean_loss2 = tr_loss2.mean() * new_diff_dist2.cdf(0)
    d_loss = (mean_loss2 - mean_loss) / dN
    res.append((N, mu, s, p, mean_loss, d_loss))
    
#res = [(N, approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)) for N in range(na+nb, 1000000, 100)]

df = pd.DataFrame(res, columns=['N', 'mu', 'std', 'p', 'mean_loss', 'd_loss'])
df


fig = go.Figure()
#fig.add_trace(go.Scatter(x=df['N'], y=df['std'] * 100, 
#                         line_color='blue', name='std * 100'))
fig.add_trace(go.Scatter(x=df['N'], y=df['p'], 
                         line_color='black', name='p'))
fig.add_trace(go.Scatter(x=df['N'], y=df['mean_loss'] * 100, 
                         line_color='red', name='mean_loss * 100'))
fig.add_trace(go.Scatter(x=df['N'], y=df['mu']*10, 
                         line_color='yellow', name='mu*10'))
fig.add_trace(go.Scatter(x=df['N'], y=df['d_loss'] * 1000000, 
                         line_color='green', name='d_loss * 1000000'))
fig.show()

Картинка апостериорных распределений и разности.  
Для 2-3 наборов точек.  
Динамика по времени.  

Критерий остановки  
Ожидаемая выгода при остановке > Ожидаемая выгодна на след. шаге - Цена шага. 

$$
E[U(\delta | N)] > E[U(\delta | N + \Delta N)] - Cost(\Delta N)
\\
E[U(\delta | N)] = LTV * M \int dx \max(0, P_{\delta(N)}(x)) = LTV * M \int_0^{\infty} P_{\delta(N)}(x) dx
\approx LTV * M \int_0^{\infty} Norm(x; \mu_B(N) - \mu_A(N), \sigma_A^2(N) + \sigma_B^2(N)) dx
\\
Cost(\Delta N) \approx  E[U(\delta | N)] \Delta N / M
$$
(учесть долю трафика в cost(dN))


$$
E[U(\delta | N)] > E[U(\delta | N + \Delta N)] - E[U(\delta | N)] \Delta N / M
\\
E[U(\delta | N)] / M > (E[U(\delta | N + \Delta N)] - E[U(\delta | N)]) / \Delta N
\\
E[U(\delta | N)] > M (E[U(\delta | N)])' 
\\
E[TrNorm(N)] > M E[TrNorm(N)]'
$$

Аналитика  
https://en.wikipedia.org/wiki/Truncated_normal_distribution  
$$
E[TrNorm(N)] = 
\\
E[TrNorm(N)]' = 
$$

Что-то не то со знаками.  
Еще вариант.  
Остановка, если улучшение оценки выгоды меньше стоимости данных.  
$$
E[U(\delta | N + \Delta N)] - E[U(\delta | N)] < Cost(\Delta N)
$$

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))

a, b = (0 - approx_diff_dist.mean()) / approx_diff_dist.std(), np.inf
tr_gain = stats.truncnorm(a=a, b=b, loc=approx_diff_dist.mean(), scale=approx_diff_dist.std())

x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='Diff'))
fig.add_trace(go.Scatter(x=x, y=tr_gain.pdf(x), 
                         line_color='red', name='Gain'))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()


# pa = sa / na
# pb = sb / nb

# for N in range(0, 10e6, 1000):
#     na_mod = na + N
#     nb_mod = nb + N
#     sa_mod = sa + int(pa * N)
#     sb_mod = sb + int(pb * N)
#     a_mod = stats.beta(a=sa_mod, b=na_mod - sa_mod)
#     b_mod = stats.beta(a=sb_mod, b=nb_mod - sb_mod)
#     diff_mod = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
#                               scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))

dN = 100

res = []
for N in range(na+nb, 100_000, dN):
    s = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)
    new_diff_dist = stats.norm(loc=approx_diff_dist.mean(), scale=s)
    p = 1 - new_diff_dist.cdf(0)
    a, b = (0 - new_diff_dist.mean()) / new_diff_dist.std(), np.inf
    tr_gain = stats.truncnorm(a=a, b=b, loc=new_diff_dist.mean(), scale=new_diff_dist.std())
    mean_gain = tr_gain.mean()
    s2 = approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N+dN)
    new_diff_dist2 = stats.norm(loc=approx_diff_dist.mean(), scale=s2)
    a2, b2 = (0 - new_diff_dist2.mean()) / new_diff_dist2.std(), np.inf
    tr_gain2 = stats.truncnorm(a=a2, b=b2, loc=new_diff_dist2.mean(), scale=new_diff_dist2.std())
    mean_gain2 = tr_gain2.mean()
    d_gain = (mean_gain2 - mean_gain) / dN
    res.append((N, s, p, mean_gain, d_gain))
    
#res = [(N, approx_diff_dist.std() * np.sqrt(na+nb) / np.sqrt(N)) for N in range(na+nb, 1000000, 100)]

df = pd.DataFrame(res, columns=['N', 'std', 'p', 'mean_gain', 'd_gain'])
df


fig = go.Figure()
fig.add_trace(go.Scatter(x=df['N'], y=df['std'] * 100, 
                         line_color='blue', name='std * 100'))
fig.add_trace(go.Scatter(x=df['N'], y=df['p'], 
                         line_color='black', name='p'))
fig.add_trace(go.Scatter(x=df['N'], y=df['mean_gain'] * 100, 
                         line_color='red', name='mean_gain * 100'))
fig.add_trace(go.Scatter(x=df['N'], y=-df['d_gain'] * 10000000, 
                         line_color='green', name='-d_gain * 10000000'))
fig.show()

In [ ]:
na = 1000
sa = 100
nb = 1000
sb = 110

p_dist_a = stats.beta(a=sa+1, b=na-sa+1)
p_dist_b = stats.beta(a=sb+1, b=nb-sb+1)

approx_diff_dist = stats.norm(loc=p_dist_b.mean() - p_dist_a.mean(), 
                              scale=np.sqrt(p_dist_b.std()**2 + p_dist_a.std()**2))
dist_p_b_gt_a = 1 - approx_diff_dist.cdf(0)

npost = 50000
samp_a = p_dist_a.rvs(size=npost)
samp_b = p_dist_b.rvs(size=npost)
samp_p_b_gt_a = np.sum(samp_b > samp_a) / npost


xaxis_max = 0.2
x = np.linspace(0, xaxis_max, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=p_dist_a.pdf(x), line_color='black', name='А'))
fig.add_trace(go.Scatter(x=x, y=p_dist_b.pdf(x), line_color='black', opacity=0.3, name='Б'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='$p$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[0, xaxis_max],
                  hovermode="x",
                  height=500)
fig.show()
#fig.write_image("./figs/ch2_conv_cmp_example.png", scale=2)
#Апостериорные распределения конверсий в обеих группах задаются бета-распределениями.

x = np.linspace(-0.3, 0.3, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=approx_diff_dist.pdf(x), 
                         line_color='black', name='$\mbox{Аналитическое приближение}$'))
fig.add_trace(go.Histogram(x=samp_b - samp_a, histnorm='probability density', 
                           name='$\mbox{Разность апостериорных выборок}$', nbinsx=500,
                           marker_color='black', opacity=0.3))
fig.add_trace(go.Scatter(x=[0, 0], y=[0, max(approx_diff_dist.pdf(x))*1.05], 
                         line_color='black', mode='lines', line_dash='dash', showlegend=False))
fig.update_layout(title='$p_B - p_A$',
                  xaxis_title='$x$',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[-0.1, 0.1],
                  hovermode="x",
                  height=500)
fig.show()
#fig.write_image("./figs/ch2_conv_cmp_diff.png", scale=2)
#Конверсия группы Б выше А с вероятностью 77%.

print(f"P(p_b > p_a) diff dist: {dist_p_b_gt_a}")
print(f"P(p_b > p_a) post samples: {samp_p_b_gt_a}")

In [ ]:
def posterior_dist_binom(ns, ntotal, a_prior=1, b_prior=1):
    a = a_prior + ns
    b = b_prior + ntotal - ns 
    return stats.beta(a=a, b=b)
    
p = 0.1
nsample = 1000

exact_dist = stats.bernoulli(p=p)
data = exact_dist.rvs(nsample)
post_dist = posterior_dist_binom(ns=np.sum(data), ntotal=len(data))

x = np.linspace(0, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_dist.pdf(x), line_color='black', name='Апостериорное'))
fig.add_trace(go.Scatter(x=[data.mean(), data.mean()], y=[0, max(post_dist.pdf(x))], 
                         line_color='black', mode='lines', line_dash='dash', name='Среднее в выборке'))
fig.add_trace(go.Scatter(x=[exact_dist.mean(), exact_dist.mean()], y=[0, max(post_dist.pdf(x))*1.05], 
                         line_color='red', mode='lines', line_dash='dash', name='Точное p'))
fig.update_layout(title='Апостериорное распределение',
                  xaxis_title='p',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[p-0.1, p+0.1],
                  hovermode="x",
                  height=500)
fig.show()
#fig.write_image("./figs/ch3_postdist.png", scale=2)
#Мода апостериорного распределения конверсии близка точному значению.

In [ ]:
def prob_pb_gt_pa(post_dist_A, post_dist_B, post_samp=100_000):
    sa = post_dist_A.rvs(size=post_samp)
    sb = post_dist_B.rvs(size=post_samp)
    b_gt_a = np.sum(sb > sa)
    return b_gt_a / post_samp

p_A = 0.1
p_B = p_A * 1.05
nsample = 1000

exact_dist_A = stats.bernoulli(p=p_A)
exact_dist_B = stats.bernoulli(p=p_B)
data_A = exact_dist_A.rvs(nsample)
data_B = exact_dist_B.rvs(nsample)

post_dist_A = posterior_dist_binom(ns=np.sum(data_A), ntotal=len(data_A))
post_dist_B = posterior_dist_binom(ns=np.sum(data_B), ntotal=len(data_B))

x = np.linspace(0, 1, 1000)
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=post_dist_A.pdf(x), line_color='black', name='A'))
fig.add_trace(go.Scatter(x=x, y=post_dist_B.pdf(x), line_color='black', opacity=0.2, name='Б'))
fig.add_trace(go.Scatter(x=[exact_dist_A.mean(), exact_dist_A.mean()], y=[0, max(post_dist_A.pdf(x))*1.05], 
                         mode='lines', line_dash='dash', line_color='black', name='Точное A'))
fig.add_trace(go.Scatter(x=[exact_dist_B.mean(), exact_dist_B.mean()], y=[0, max(post_dist_A.pdf(x))*1.05], 
                         mode='lines', line_dash='dash', line_color='black', opacity=0.2, name='Точное Б'))
fig.update_layout(title='Апостериорные распределения',
                  xaxis_title='p',
                  yaxis_title='Плотность вероятности',
                  xaxis_range=[p_A/2, p_A*2],
                  hovermode="x",
                  height=500)
fig.show()
#fig.write_image("./figs/ch3_groups_cmp.png", scale=2)
#Апостериорные распределения конверсии в группах. Конверсия группы Б выше А с вероятностью 84%.

print(f'P(pB > pA): {prob_pb_gt_pa(post_dist_A, post_dist_B)}')

Оценка при продолжении.  
Вначале посчитать распределение ожидаемых новых точек.  
Это апострериорное предиктивное распределение.  
Потом использовать это распределение как функцию правдоподобия в соотн. Байеса.



https://en.wikipedia.org/wiki/Conjugate_prior

https://en.wikipedia.org/wiki/Posterior_predictive_distribution

https://en.wikipedia.org/wiki/Beta-binomial_distribution

Апостериорное предиктивное для биномиального и бета - бета-биномиальное.

Вероятность, что будет еще 1 успех:

$$
P(N_2, n_{s_2} | N, n_s) = \int dp Binom(n_{s_2}, N_2; p) Beta(p; \alpha + n_s, \beta + N - n_s)
\\
=  C^{n_{s_2}}_{N_2} \frac{1}{B(n_s + \alpha - 1, N - n_s + \beta - 1)} \int dp p^{n_{s_2}} (1-p)^{N_2-n_{s_2}} p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}
\\
=  C^{n_{s_2}}_{N_2} 
\frac{B(n_{s_2} + n_s + \alpha - 1, N_2-n_{s_2} + N - n_s + \beta - 1)}{B(n_s + \alpha - 1, N - n_s + \beta - 1)} 
$$

После интегрирования по p перестает зависеть от p.  
Как обновлять оценки параметров не ясно.

$$
\begin{split}
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) & = \mbox{Binom}(n_s, N; p) = C^{n_s}_{N} p^{n_s} (1 - p)^{N-n_s}
\\
\\
P(\mathcal{H}) = P(p) & = \mbox{Unif}(0, 1) = 1
\\
\\
P(\mathcal{H} | \mathcal{D}) = P(p | n_s, N) 
& = \frac{P(n_s, N | p) P(p)}{P(n_s, N)}
= \frac{P(n_s, N | p) P(p)}{\int_0^1 d p P(n_s, N | p) P(p)}
\\
& = \frac{p^{n_s} (1 - p)^{N-n_s}}{\int_0^1 d p (1 - p)^{N-n_s} p^{n_s} }
= \mbox{Beta}(p; n_s + 1, N - n_s + 1)
\\
\\
\mbox{Beta}(x; \alpha, \beta) & \equiv \frac{x^{\alpha-1} (1 - x)^{\beta-1}}{\int_0^1 dx x^{\alpha-1} (1 - x)^{\beta-1}}
 = \frac{\Gamma(\alpha+\beta)}{\Gamma(\alpha)\Gamma(\beta)} x^{\alpha-1} (1 - x)^{\beta-1}
\end{split}
$$

$$
P(\mathcal{H} | \mathcal{D}) \propto P(\mathcal{D} | \mathcal{H}) P(\mathcal{H})
$$

$$
P(\mathcal{D} | \mathcal{H}) = P(n_s, N | p) = \mbox{Binom}(n_s, N | p) = C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
$$

$$
P(\mathcal{H}) = P(p) = \mbox{Beta}(p; \alpha, \beta) = 
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
$$

$$
\begin{split}
P(\mathcal{H} | \mathcal{D}) & = P(p | n_s, N) 
\\
& \propto \mbox{Binom}(n_s, N | p) \mbox{Beta}(p; \alpha, \beta)
\\
& \propto C_{N}^{n_s} p^{n_s} (1-p)^{N-n_s}
\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha) \Gamma(\beta)} p^{\alpha-1}(1-p)^{\beta-1}
\\
& \propto p^{n_s + \alpha - 1} (1-p)^{N - n_s + \beta - 1}
\\
& = \mbox{Beta}(p; \alpha + n_s, \beta + N - n_s)
\end{split}
$$